In [6]:
import logging
from collections import deque
from pprint import pprint
import numpy as np

logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s', datefmt='%H:%M:%S', level=logging.INFO)

In [5]:
def _contains_duplicates(X):
    return np.sum(np.unique(X)) != np.sum(X)

def contains_duplicates(sol):
    return any(_contains_duplicates(sol[r,:]) for r in range(9)) or \
           any(_contains_duplicates(sol[:,r]) for r in range(9)) or \
           any(_contains_duplicates(sol[r:r+3:,c:c+3]) for r in range(0,9,3) for c in range(0,9,3))

def check_duplicates(node, x, y, value):
    new_node = node.copy()
    new_node[x, y] = value
    return contains_duplicates(new_node)

def valid_solution(sol):
    return not contains_duplicates(sol) and np.sum(sol) == (1+2+3+4+5+6+7+8+9) * 9

def print_sudoku(sudoku):
    print("+-------+-------+-------+")
    for b in range(0, 9, 3):
        for r in range(3):
            print("|", " | ".join(" ".join(str(_) for _ in sudoku[b+r, c:c+3]) for c in range(0, 9, 3)), "|")
        print("+-------+-------+-------+")

In [46]:
def COUNTER(possible_nodes):
    return possible_nodes[3]

In [61]:
def dfsolve(sudoku):
    """Vanilla depth-first solver for sudoku puzzles"""
    frontier = deque([sudoku.copy()])
    num_nodes = 0
    while frontier:
        node = frontier.popleft()
        num_nodes += 1

        if valid_solution(node):
            logging.info(f"Solved after expanding {num_nodes:,} nodes")
            return node

        possible_nodes = []
        for i, j in zip(*np.where(node == 0)):
            counter = 0
            for c in range(1, 10):
                if not check_duplicates(node, i, j, c):
                    counter += 1
            possible_nodes.append((node, i, j, counter))

        possible_nodes = sorted(possible_nodes, key=lambda nodes : nodes[3])
        
        for p, i, j, _ in possible_nodes:
            for c in range(1, 10):
                p[i, j] = c
                if not contains_duplicates(p):
                    frontier.appendleft(p.copy())
                      
    logging.info(f"Giving up after expanding {num_nodes:,} nodes")
    return None

In [17]:
simple_sudoku = np.array([[6, 0, 4,    0, 7, 0,    0, 0, 1],
                          [0, 5, 0,    0, 0, 0,    0, 7, 0], 
                          [7, 0, 0,    5, 9, 6,    8, 3, 4], 
       
                          [0, 8, 0,    0, 0, 2,    4, 9, 0], 
                          [1, 0, 0,    0, 0, 0,    0, 0, 3], 
                          [0, 6, 9,    7, 0, 0,    0, 5, 0], 
       
                          [9, 1, 8,    3, 6, 7,    0, 0, 5], 
                          [0, 4, 0,    0, 0, 0,    0, 6, 0], 
                          [2, 0, 0,    0, 5, 0,    7, 0, 8]], dtype=np.int8)

In [12]:
def sudoku_generator(sudokus=1, *, kappa=5, random_seed=None):
    if random_seed:
        np.random.seed(random_seed)
    for puzzle in range(sudokus):
        sudoku = np.zeros((9, 9), dtype=np.int8)
        for cell in range(np.random.randint(kappa)):        #range tra 0 e il numero casuale generato tra 0 e 5
            for p, val in zip(np.random.randint(0, 8, size=(9, 2)), range(1, 10)):
                tmp = sudoku.copy()
                sudoku[tuple(p)] = val
                if contains_duplicates(sudoku):
                    sudoku = tmp
        yield sudoku.copy()

In [65]:
for sudoku in sudoku_generator(random_seed=44):
    print_sudoku(sudoku)
    solution = dfsolve(sudoku)
    if solution is not None:
        print_sudoku(solution)

+-------+-------+-------+
| 0 7 0 | 0 0 0 | 0 6 0 |
| 0 4 0 | 8 0 0 | 0 2 0 |
| 6 5 0 | 0 9 0 | 0 0 0 |
+-------+-------+-------+
| 8 1 0 | 0 0 0 | 0 0 0 |
| 0 0 3 | 4 0 1 | 0 0 0 |
| 0 0 0 | 2 0 7 | 0 0 0 |
+-------+-------+-------+
| 0 0 0 | 0 0 0 | 0 0 0 |
| 0 0 5 | 9 6 0 | 7 0 0 |
| 0 0 0 | 0 0 0 | 0 0 0 |
+-------+-------+-------+


[17:36:40] INFO: Solved after expanding 57 nodes


+-------+-------+-------+
| 2 7 8 | 3 4 5 | 9 6 1 |
| 3 4 9 | 8 1 6 | 5 2 7 |
| 6 5 1 | 7 9 2 | 8 3 4 |
+-------+-------+-------+
| 8 1 2 | 6 5 9 | 4 7 3 |
| 7 9 3 | 4 8 1 | 6 5 2 |
| 5 6 4 | 2 3 7 | 1 8 9 |
+-------+-------+-------+
| 9 8 7 | 5 2 4 | 3 1 6 |
| 1 2 5 | 9 6 3 | 7 4 8 |
| 4 3 6 | 1 7 8 | 2 9 5 |
+-------+-------+-------+
